In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np
from model import Autoencoder
transform = transforms.ToTensor()
train_dataset = datasets.MNIST(root='./data', train=False, download=False)
# 1. Define the Autoencoder Model


model = torch.compile(Autoencoder(5))
model.load_state_dict(torch.load("./models/autoencoder_mnist_80.pth"))
model = model._orig_mod
from PIL import Image

def numpy_to_pil_image(np_array):
    """
    Converts a NumPy array with shape (1, H, W) to a PIL Image.
    Handles both integer (uint8) and float arrays.

    Args:
        np_array (np.ndarray): The input NumPy array. Can be of shape (1, H, W)
                               or (H, W). Supports np.uint8 and float data types.

    Returns:
        PIL.Image.Image: The resulting grayscale PIL Image.
    """
    # Check for the (1, H, W) shape and squeeze if necessary
    if np_array.ndim == 3 and np_array.shape[0] == 1:
        np_array = np.squeeze(np_array, axis=0)
    
    # Check if the array is a float type and scale it
    if np.issubdtype(np_array.dtype, np.floating):
        # Assuming float values are in the [0.0, 1.0] range
        np_array = (np_array * 255).astype(np.uint8)
    elif np.issubdtype(np_array.dtype, np.integer):
        # Assuming integer values are already in the [0, 255] range
        np_array = np_array.astype(np.uint8)
    else:
        raise ValueError(f"Unsupported data type: {np_array.dtype}")
    
    pil_image = Image.fromarray(np_array, mode='L')
    return pil_image

with torch.no_grad(): # Disables gradient computation
    for i in range(10):
        data = train_dataset[i]
        img, _ = data
        img_t = transform(img)
        old_shape = img_t.shape
        img_t = img_t.view(img_t.size(0), -1)
        output = model(img_t)
        output = output.view(old_shape)
        output = numpy_to_pil_image(output.cpu().numpy())
        display(img) 
        display(output)

/tmp/ipykernel_893194/3257515201.py:44: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  pil_image = Image.fromarray(np_array, mode='L')
